In [ ]:
# code ที่ไว้ทำ model
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model, load_model, Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, LambdaCallback
from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.utils.class_weight import compute_class_weight
import librosa
import librosa.display
import joblib
import pickle
import cv2
import warnings
warnings.filterwarnings('ignore')
def extract_advanced_features(audio_path, max_length=6.0):
    """
    สกัดคุณลักษณะที่หลากหลายจากไฟล์เสียง
    """
    try:
        # โหลดไฟล์เสียง
        y, sr = librosa.load(audio_path, sr=22050)
        
        # ปรับความยาวเสียงให้เท่ากัน
        target_length = int(max_length * sr)
        
        if len(y) > target_length:
            y = y[:target_length]
        else:
            # Pad with zeros
            padding = target_length - len(y)
            y = np.pad(y, (0, padding), 'constant')
        
        # 1. Mel Spectrogram - ให้ความสำคัญมากที่สุด
        mel_spec = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128, 
                                                fmax=8000, n_fft=2048, hop_length=512)
        mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max)
        
        # 2. MFCC - Mel-frequency cepstral coefficients
        mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=20)
        delta_mfcc = librosa.feature.delta(mfcc)
        delta2_mfcc = librosa.feature.delta(mfcc, order=2)
        
        # รวม features แบบมัลติ-แชนแนล (3 ช่อง)
        # Normalize features
        mel_spec_norm = (mel_spec_db - np.mean(mel_spec_db)) / (np.std(mel_spec_db) + 1e-10)
        mfcc_norm = (mfcc - np.mean(mfcc)) / (np.std(mfcc) + 1e-10)
        delta_norm = (delta_mfcc - np.mean(delta_mfcc)) / (np.std(delta_mfcc) + 1e-10)
        
        # ปรับขนาดให้เท่ากัน
        target_shape = (128, 128)
        mel_spec_resized = cv2.resize(mel_spec_norm, (target_shape[1], target_shape[0]))
        mfcc_resized = cv2.resize(mfcc_norm, (target_shape[1], target_shape[0]))
        delta_resized = cv2.resize(delta_norm, (target_shape[1], target_shape[0]))
        
        # รวมเป็นภาพ 3 ช่อง (RGB-like)
        feature_image = np.stack([mel_spec_resized, mfcc_resized, delta_resized], axis=-1)
        
        # ทำให้ค่าอยู่ระหว่าง 0-1
        feature_image = (feature_image - feature_image.min()) / (feature_image.max() - feature_image.min() + 1e-10)
        
        return feature_image
    except Exception as e:
        print(f"Error extracting features from {audio_path}: {str(e)}")
        return None

def load_and_preprocess_data(data_dir, selected_accents, max_samples=150):
    X = []
    y = []
    file_paths = []
    
    print("กำลังโหลดข้อมูลเสียง...")
    print(f"โฟลเดอร์ข้อมูล: {data_dir}")
    
    # ตรวจสอบว่าโฟลเดอร์ข้อมูลมีอยู่จริง
    if not os.path.exists(data_dir):
        print(f"ไม่พบโฟลเดอร์ข้อมูล: {data_dir}")
        return None, None, None, None
    
    # สร้าง Label Encoder
    label_encoder = LabelEncoder()
    label_encoder.fit(selected_accents)
    
    # โหลดข้อมูลจากแต่ละสำเนียงที่เลือก
    total_files_found = 0
    for accent in selected_accents:
        accent_dir = os.path.join(data_dir, accent)
        if not os.path.isdir(accent_dir):
            print(f"ไม่พบโฟลเดอร์สำหรับสำเนียง {accent} ที่ {accent_dir}")
            continue
        
        accent_files = [os.path.join(accent_dir, f) for f in os.listdir(accent_dir) 
                        if f.endswith('.wav') or f.endswith('.mp3')]
        
        total_files_found += len(accent_files)
        
        # จำกัดจำนวนตัวอย่างต่อสำเนียง
        if max_samples and len(accent_files) > max_samples:
            accent_files = accent_files[:max_samples]
        
        print(f"กำลังประมวลผลสำเนียง {accent}: พบ {len(accent_files)} ไฟล์")
        
        files_processed = 0
        for file_path in accent_files:
            features = extract_advanced_features(file_path)
            if features is not None:
                X.append(features)
                y.append(accent)
                file_paths.append(file_path)
                files_processed += 1
        
        print(f"  - ประมวลผลสำเร็จ {files_processed} ไฟล์")
    
    print(f"พบไฟล์เสียงทั้งหมด: {total_files_found}")
    print(f"ประมวลผลสำเร็จ: {len(X)} ไฟล์")
    
    if len(X) == 0:
        print("ไม่มีข้อมูลที่ประมวลผลได้ โปรดตรวจสอบข้อมูลเสียงของคุณ")
        return None, None, None, None
    
    # แปลง list เป็น numpy array
    X = np.array(X)
    
    # แปลงป้ายกำกับเป็น one-hot encoding
    y_encoded = label_encoder.transform(y)
    y_one_hot = to_categorical(y_encoded)
    
    print(f"โหลดข้อมูลเสร็จสิ้น: {X.shape[0]} ตัวอย่าง, คุณลักษณะ shape: {X.shape[1:]}")
    
    return X, y_one_hot, label_encoder, file_paths
def build_cnn_rnn_model(input_shape, num_classes):
    """
    สร้างโมเดลแบบผสมผสาน CNN + RNN สำหรับการตรวจจับสำเนียง
    """
    # Input layer
    inputs = Input(shape=input_shape)
    
    # CNN Blocks - สกัดคุณลักษณะเชิงพื้นที่
    x = Conv2D(32, (3, 3), padding='same', activation='relu')(inputs)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    
    x = Conv2D(64, (3, 3), padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = Conv2D(64, (3, 3), padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    
    x = Conv2D(128, (3, 3), padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = Conv2D(128, (3, 3), padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    
    # Reshape เพื่อให้เข้ากับ RNN
    reshape_dim = K.int_shape(x)
    x = Reshape((reshape_dim[1], reshape_dim[2] * reshape_dim[3]))(x)
    
    # RNN Layers - จับลักษณะเชิงลำดับเวลา
    x = Bidirectional(GRU(128, return_sequences=True))(x)
    x = Dropout(0.3)(x)
    x = Bidirectional(GRU(128))(x)
    x = Dropout(0.3)(x)
    
    # Global Features
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.5)(x)
    outputs = Dense(num_classes, activation='softmax')(x)
    
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(
        optimizer=Adam(learning_rate=0.0003),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model
def residual_block(x, filters, kernel_size=3):
    """
    สร้างบล็อก Residual สำหรับโมเดล ResNet
    """
    # แบบ Residual Block ช่วยให้เทรนโมเดลลึกได้มากขึ้น
    y = Conv2D(filters, kernel_size, padding='same')(x)
    y = BatchNormalization()(y)
    y = Activation('relu')(y)
    y = Conv2D(filters, kernel_size, padding='same')(y)
    y = BatchNormalization()(y)
    
    # สร้าง shortcut connection
    if K.int_shape(x)[-1] != filters:
        x = Conv2D(filters, 1, padding='same')(x)
        x = BatchNormalization()(x)
    
    out = Add()([x, y])
    out = Activation('relu')(out)
    return out

def build_resnet_model(input_shape, num_classes):
    """
    สร้างโมเดลแบบ ResNet สำหรับการตรวจจับสำเนียง
    """
    # Input layer
    inputs = Input(shape=input_shape)
    
    # Initial Conv Layer
    x = Conv2D(32, 7, strides=2, padding='same')(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(3, strides=2, padding='same')(x)
    
    # Residual blocks
    x = residual_block(x, 32)
    x = residual_block(x, 32)
    x = MaxPooling2D(2, strides=2, padding='same')(x)
    
    x = residual_block(x, 64)
    x = residual_block(x, 64)
    x = MaxPooling2D(2, strides=2, padding='same')(x)
    
    x = residual_block(x, 128)
    x = residual_block(x, 128)
    
    # Final layers
    x = GlobalAveragePooling2D()(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.3)(x)
    outputs = Dense(num_classes, activation='softmax')(x)
    
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(
        optimizer=Adam(learning_rate=0.0003),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model
def positional_encoding(position, d_model):
    """
    สร้าง positional encoding สำหรับ Transformer
    """
    def get_angles(pos, i, d_model):
        angle_rates = 1 / np.power(10000, (2 * (i//2)) / np.float32(d_model))
        return pos * angle_rates
    
    angle_rads = get_angles(np.arange(position)[:, np.newaxis],
                            np.arange(d_model)[np.newaxis, :],
                            d_model)
    
    # sin ใช้สำหรับตำแหน่งคู่ (0, 2, 4, ...)
    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])
    
    # cos ใช้สำหรับตำแหน่งคี่ (1, 3, 5, ...)
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])
    
    pos_encoding = angle_rads[np.newaxis, ...]
    
    return tf.cast(pos_encoding, dtype=tf.float32)

def scaled_dot_product_attention(q, k, v, mask=None):
    """
    คำนวณ scaled dot product attention
    """
    matmul_qk = tf.matmul(q, k, transpose_b=True)
    
    # scale matmul_qk
    dk = tf.cast(tf.shape(k)[-1], tf.float32)
    scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)
    
    # add mask (optional)
    if mask is not None:
        scaled_attention_logits += (mask * -1e9)  
    
    # softmax normalization
    attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)
    
    output = tf.matmul(attention_weights, v)
    
    return output, attention_weights

class MultiHeadAttention(Layer):
    """
    Multi-head attention layer
    """
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model
        
        assert d_model % self.num_heads == 0
        
        self.depth = d_model // self.num_heads
        
        self.wq = Dense(d_model)
        self.wk = Dense(d_model)
        self.wv = Dense(d_model)
        
        self.dense = Dense(d_model)
    
    def split_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(x, perm=[0, 2, 1, 3])
    
    def call(self, v, k, q, mask=None):
        batch_size = tf.shape(q)[0]
        
        q = self.wq(q)
        k = self.wk(k)
        v = self.wv(v)
        
        q = self.split_heads(q, batch_size)
        k = self.split_heads(k, batch_size)
        v = self.split_heads(v, batch_size)
        
        scaled_attention, attention_weights = scaled_dot_product_attention(
            q, k, v, mask)
        
        scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])
        
        concat_attention = tf.reshape(scaled_attention, 
                                        (batch_size, -1, self.d_model))
        
        output = self.dense(concat_attention)
        
        return output

def point_wise_feed_forward_network(d_model, dff):
    """
    Simple feed forward network for transformer
    """
    return Sequential([
        Dense(dff, activation='relu'),
        Dense(d_model)
    ])

class EncoderLayer(Layer):
    """
    Encoder layer for transformer
    """
    def __init__(self, d_model, num_heads, dff, rate=0.1):
        super(EncoderLayer, self).__init__()
        
        self.mha = MultiHeadAttention(d_model, num_heads)
        self.ffn = point_wise_feed_forward_network(d_model, dff)
        
        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)
        
        self.dropout1 = Dropout(rate)
        self.dropout2 = Dropout(rate)
    
    def call(self, x, training=True):
        attn_output = self.mha(x, x, x)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(x + attn_output)
        
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        out2 = self.layernorm2(out1 + ffn_output)
        
        return out2

def build_transformer_model(input_shape, num_classes):
    """
    สร้างโมเดล Transformer-based สำหรับการตรวจจับสำเนียง
    """
    # Input layer
    inputs = Input(shape=input_shape)
    
    # ใช้ CNN เพื่อสกัดคุณลักษณะ
    x = Conv2D(32, 3, padding='same', activation='relu')(inputs)
    x = BatchNormalization()(x)
    x = MaxPooling2D(2)(x)
    
    x = Conv2D(64, 3, padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(2)(x)
    
    x = Conv2D(128, 3, padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(2)(x)
    
    # Reshape สำหรับ Transformer
    shape_before_flatten = K.int_shape(x)
    x = Reshape((shape_before_flatten[1] * shape_before_flatten[2], shape_before_flatten[3]))(x)
    
    # โปรเจคชั่นเป็นมิติของ Transformer
    d_model = 128
    x = Dense(d_model)(x)
    
    # เพิ่ม positional encoding
    seq_len = shape_before_flatten[1] * shape_before_flatten[2]
    pos_encoding = positional_encoding(seq_len, d_model)
    x = x + pos_encoding[:, :seq_len, :]
    
    # Dropout
    x = Dropout(0.2)(x)
    
    # Transformer encoder layers
    num_layers = 2
    for i in range(num_layers):
        x = EncoderLayer(d_model=d_model, num_heads=4, dff=256, rate=0.2)(x)
    
    # Global pooling
    x = GlobalAveragePooling1D()(x)
    
    # Final layers
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.3)(x)
    outputs = Dense(num_classes, activation='softmax')(x)
    
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(
        optimizer=Adam(learning_rate=0.0001),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model
def train_model_with_advanced_techniques(model, X_train, y_train, X_val, y_val, epochs=200, batch_size=32, model_path='model.h5'):
    """
    ฝึกสอนโมเดลด้วยเทคนิคขั้นสูงเพื่อเพิ่มประสิทธิภาพ
    """
    # ระบบ Callbacks ที่หลากหลาย
    callbacks = [
        # บันทึกโมเดลที่ดีที่สุด
        ModelCheckpoint(
            filepath=model_path,
            monitor='val_accuracy',
            save_best_only=True,
            verbose=1
        ),
        # ปรับลด learning rate เมื่อประสิทธิภาพไม่ดีขึ้น
        ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=10,
            min_lr=1e-6,
            verbose=1
        ),
        # หยุดการฝึกสอนเมื่อไม่มีการพัฒนา
        EarlyStopping(
            monitor='val_accuracy',
            patience=30,
            restore_best_weights=True,
            verbose=1
        ),
        # บันทึก learning rate ในแต่ละ epoch
        LambdaCallback(
            on_epoch_end=lambda epoch, logs: logs.update({'learning_rate': float(K.get_value(model.optimizer.learning_rate))})
        )
    ]
    
    # Class weight balancing สำหรับข้อมูลที่ไม่สมดุล
    # คำนวณน้ำหนักของแต่ละคลาส
    class_weights = compute_class_weight(
        'balanced',
        classes=np.unique(np.argmax(y_train, axis=1)),
        y=np.argmax(y_train, axis=1)
    )
    class_weight_dict = {i: class_weights[i] for i in range(len(class_weights))}
    
    # ฝึกสอนโมเดล
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=epochs,
        batch_size=batch_size,
        callbacks=callbacks,
        class_weight=class_weight_dict,
        verbose=1
    )
    
    # โหลดโมเดลที่ดีที่สุด
    best_model = load_model(model_path)
    
    return history, best_model
def evaluate_and_visualize(model, X_test, y_test, class_names, label_encoder, history=None, is_ensemble=False):
    """
    ประเมินโมเดลและแสดงผลด้วยภาพ
    """
    # ทำนายด้วยชุดข้อมูลทดสอบ
    if is_ensemble:
        y_pred_probs = ensemble_predict(model, X_test)
    else:
        # ประเมินโมเดล
        test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=1)
        print(f"\nความแม่นยำบนชุดข้อมูลทดสอบ: {test_accuracy * 100:.2f}%")
        y_pred_probs = model.predict(X_test)
    
    y_pred = np.argmax(y_pred_probs, axis=1)
    y_true = np.argmax(y_test, axis=1)
    
    # แสดงผลการประเมิน
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    f1 = f1_score(y_true, y_pred, average='weighted')
    
    print("\n===== รายงานประสิทธิภาพโมเดล =====")
    print(f"Accuracy: {accuracy * 100:.2f}%")
    print(f"Precision: {precision * 100:.2f}%")
    print(f"Recall: {recall * 100:.2f}%")
    print(f"F1 Score: {f1 * 100:.2f}%")
    
    print("\n===== รายงานการจำแนกประเภทโดยละเอียด =====")
    print(classification_report(y_true, y_pred, target_names=class_names))
    
    # แสดง Confusion Matrix ด้วย Seaborn
    plt.figure(figsize=(14, 12))
    cm = confusion_matrix(y_true, y_pred)
    
    # แสดงทั้งจำนวนและเปอร์เซ็นต์
    cm_percent = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis] * 100
    
    # สร้าง custom annotation
    annot = np.empty_like(cm, dtype=str)
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            annot[i, j] = f'{cm[i, j]}\n({cm_percent[i, j]:.1f}%)'
    
    ax = sns.heatmap(cm, annot=cm, fmt='', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
    
    # ปรับปรุงความสวยงาม
    plt.title('Confusion Matrix', fontsize=18, pad=20)
    plt.xlabel('Predicted Label', fontsize=14)
    plt.ylabel('True Label', fontsize=14)
    plt.xticks(rotation=45, ha='right', fontsize=12)
    plt.yticks(fontsize=12)
    plt.tight_layout()
    plt.savefig('confusion_matrix.png', dpi=300)
    plt.show()
    
    # แสดงกราฟประวัติการฝึกสอน
    if history:
        # กราฟความแม่นยำและ loss
        plt.figure(figsize=(16, 6))
        
        # กราฟความแม่นยำ
        plt.subplot(1, 2, 1)
        plt.plot(history.history['accuracy'], 'o-', linewidth=2, label='Train Accuracy')
        plt.plot(history.history['val_accuracy'], 'o-', linewidth=2, label='Validation Accuracy')
        plt.title('Model Accuracy Over Time', fontsize=16)
        plt.xlabel('Epoch', fontsize=14)
        plt.ylabel('Accuracy', fontsize=14)
        plt.grid(True, linestyle='--', alpha=0.6)
        plt.legend(fontsize=12)
        
        # กราฟค่าสูญเสีย
        plt.subplot(1, 2, 2)
        plt.plot(history.history['loss'], 'o-', linewidth=2, label='Train Loss')
        plt.plot(history.history['val_loss'], 'o-', linewidth=2, label='Validation Loss')
        plt.title('Model Loss Over Time', fontsize=16)
        plt.xlabel('Epoch', fontsize=14)
        plt.ylabel('Loss', fontsize=14)
        plt.grid(True, linestyle='--', alpha=0.6)
        plt.legend(fontsize=12)
        
        plt.tight_layout()
        plt.savefig('training_history.png', dpi=300)
        plt.show()
        
        # กราฟการเปรียบเทียบความแม่นยำรายคลาส
        plt.figure(figsize=(12, 10))
        
        # คำนวณความแม่นยำรายคลาส
        class_accuracy = {}
        for i, name in enumerate(class_names):
            # เลือกเฉพาะตัวอย่างที่เป็นคลาสนั้น
            class_indices = np.where(y_true == i)[0]
            class_true = y_true[class_indices]
            class_pred = y_pred[class_indices]
            class_accuracy[name] = accuracy_score(class_true, class_pred) if len(class_indices) > 0 else 0
        
        # เรียงลำดับคลาสตามความแม่นยำ
        sorted_classes = sorted(class_accuracy.items(), key=lambda x: x[1], reverse=True)
        
        # สร้างกราฟแท่ง
        bars = plt.bar(
            [name.capitalize() for name, _ in sorted_classes],
            [acc * 100 for _, acc in sorted_classes],
            color=[plt.cm.Blues(0.5 + 0.5 * i / len(class_names)) for i in range(len(class_names))]
        )
        
        # เพิ่มค่าบนแท่ง
        for bar in bars:
            height = bar.get_height()
            plt.text(
                bar.get_x() + bar.get_width()/2.,
                height + 1,
                f'{height:.1f}%',
                ha='center', va='bottom', fontsize=12
            )
        
        plt.title
        plt.title('Accuracy by Language/Accent', fontsize=16)
        plt.xlabel('Language', fontsize=14)
        plt.ylabel('Accuracy (%)', fontsize=14)
        plt.ylim(0, 105)
        plt.grid(axis='y', linestyle='--', alpha=0.6)
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()
        plt.savefig('accuracy_by_language.png', dpi=300)
        plt.show()
    
    # แสดงตัวอย่างการทำนาย
    num_samples = min(5, len(X_test))
    sample_indices = np.random.choice(len(X_test), num_samples, replace=False)
    
    print("\n===== ตัวอย่างการทำนาย =====")
    for i, idx in enumerate(sample_indices):
        true_class = class_names[np.argmax(y_test[idx])]
        pred_probs = y_pred_probs[idx]
        
        # จัดอันดับการทำนาย
        top_indices = np.argsort(pred_probs)[::-1][:3]
        
        print(f"\nตัวอย่างที่ {i+1}:")
        print(f"สำเนียงจริง: {true_class}")
        
        for j, index in enumerate(top_indices):
            print(f"อันดับ {j+1}: {class_names[index]} - {pred_probs[index]*100:.1f}%")
        def build_ensemble_models(X, y, num_models=2):  # ลดจำนวนโมเดลจาก 3 เป็น 2
    """
    สร้าง ensemble ของโมเดลหลายตัวเพื่อเพิ่มประสิทธิภาพการทำนาย
    """
    models = []
    input_shape = X.shape[1:]
    num_classes = y.shape[1]
    
    for i in range(num_models):
        print(f"กำลังสร้างโมเดลที่ {i+1}/{num_models}...")
        
        if i == 0:
            # โมเดลแรกใช้ CNN+RNN Hybrid
            model = build_cnn_rnn_model(input_shape, num_classes)
        else:
            # โมเดลที่สองใช้ ResNet
            model = build_resnet_model(input_shape, num_classes)
        
        models.append(model)
    
    return models

def train_ensemble_models(models, X_train, y_train, X_val, y_val, base_model_path='Model/accent_model'):
    """
    ฝึกสอนโมเดล ensemble แต่ละตัว
    """
    trained_models = []
    histories = []
    
    for i, model in enumerate(models):
        print(f"กำลังฝึกสอนโมเดลที่ {i+1}/{len(models)}...")
        model_path = f"{base_model_path}_{i+1}.h5"
        
        # ฝึกสอนโมเดล
        history, trained_model = train_model_with_advanced_techniques(
            model, X_train, y_train, X_val, y_val,
            epochs=200, batch_size=32, model_path=model_path
        )
        
        trained_models.append(trained_model)
        histories.append(history)
    
    return trained_models, histories

def ensemble_predict(models, X):
    """
    ทำนายโดยใช้ ensemble ของโมเดลหลายตัว
    """
    predictions = []
    
    for model in models:
        pred = model.predict(X)
        predictions.append(pred)
    
    # รวมการทำนายจากทุกโมเดลโดยใช้ค่าเฉลี่ย
    ensemble_pred = np.mean(predictions, axis=0)
    
    return ensemble_pred
def main():
    # กำหนดค่าเริ่มต้น
    data_dir = r"D:\Y2.2\Speech_Accent_Detection\main_datadset"  # แก้ไขเป็นโฟลเดอร์ที่เก็บข้อมูลเสียง
    model_dir = "Model"
    
    # สร้างโฟลเดอร์สำหรับเก็บโมเดล
    os.makedirs(model_dir, exist_ok=True)
    
    # เลือกภาษาที่มีลักษณะเสียงแตกต่างกันชัดเจน
    selected_accents = ['thai', 'english', 'mandarin', 'arabic', 'japanese', 'hindi']
    
    # โหลดข้อมูลและเตรียมพร้อม
    result = load_and_preprocess_data(data_dir, selected_accents)
    
    if result[0] is None:
        print("ไม่สามารถดำเนินการต่อได้เนื่องจากไม่มีข้อมูล")
        return
    
    X, y, label_encoder, file_paths = result
    
    # ดำเนินการต่อไป...  
    # บันทึก label encoder ไว้ใช้ในอนาคต
    joblib.dump(label_encoder, os.path.join(model_dir, 'label_encoder.pkl'))
    
    # แบ่งข้อมูลเป็นชุดฝึกสอน, ชุดตรวจสอบ, และชุดทดสอบ
    X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=np.argmax(y, axis=1))
    X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.2, random_state=42, stratify=np.argmax(y_train_val, axis=1))
    
    print(f"ขนาดข้อมูลฝึกสอน: {X_train.shape[0]} ตัวอย่าง")
    print(f"ขนาดข้อมูลตรวจสอบ: {X_val.shape[0]} ตัวอย่าง")
    print(f"ขนาดข้อมูลทดสอบ: {X_test.shape[0]} ตัวอย่าง")
    
    # ฝึกสอนโมเดล CNN + RNN hybrid
    print("\n===== กำลังฝึกสอนโมเดล CNN + RNN hybrid =====")
    cnn_rnn_model = build_cnn_rnn_model(X_train.shape[1:], y_train.shape[1])
    cnn_rnn_history, trained_cnn_rnn_model = train_model_with_advanced_techniques(
        cnn_rnn_model, X_train, y_train, X_val, y_val, 
        epochs=200, batch_size=32, model_path=os.path.join(model_dir, 'accent_cnn_rnn_model.h5')
    )
    
    # ประเมินผลโมเดล CNN + RNN hybrid
    print("\n===== ผลลัพธ์การทดสอบโมเดล CNN + RNN hybrid =====")
    class_names = label_encoder.classes_
    evaluate_and_visualize(trained_cnn_rnn_model, X_test, y_test, class_names, label_encoder, cnn_rnn_history)
    
    # ฝึกสอนโมเดล ResNet
    print("\n===== กำลังฝึกสอนโมเดล ResNet =====")
    resnet_model = build_resnet_model(X_train.shape[1:], y_train.shape[1])
    resnet_history, trained_resnet_model = train_model_with_advanced_techniques(
        resnet_model, X_train, y_train, X_val, y_val, 
        epochs=200, batch_size=32, model_path=os.path.join(model_dir, 'accent_resnet_model.h5')
    )
    
    # ประเมินผลโมเดล ResNet
    print("\n===== ผลลัพธ์การทดสอบโมเดล ResNet =====")
    evaluate_and_visualize(trained_resnet_model, X_test, y_test, class_names, label_encoder, resnet_history)
    
    # สร้างและฝึกสอนโมเดล Ensemble
    print("\n===== กำลังสร้างและฝึกสอนโมเดล Ensemble =====")
    ensemble_models = build_ensemble_models(X_train, y_train, num_models=2)
    trained_ensemble, ensemble_histories = train_ensemble_models(
        ensemble_models, X_train, y_train, X_val, y_val, 
        base_model_path=os.path.join(model_dir, 'accent_ensemble_model')
    )
    
    # ประเมินผลโมเดล Ensemble
    print("\n===== ผลลัพธ์การทดสอบโมเดล Ensemble =====")
    # ทำนายด้วย ensemble
    ensemble_preds = ensemble_predict(trained_ensemble, X_test)
    y_pred = np.argmax(ensemble_preds, axis=1)
    y_true = np.argmax(y_test, axis=1)
    
    # แสดงผลการประเมิน
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    f1 = f1_score(y_true, y_pred, average='weighted')
    
    print("\n===== รายงานประสิทธิภาพโมเดล Ensemble =====")
    print(f"Accuracy: {accuracy * 100:.2f}%")
    print(f"Precision: {precision * 100:.2f}%")
    print(f"Recall: {recall * 100:.2f}%")
    print(f"F1 Score: {f1 * 100:.2f}%")
    
    print("\n===== รายงานการจำแนกประเภทโดยละเอียด =====")
    print(classification_report(y_true, y_pred, target_names=class_names))
    
    # แสดง Confusion Matrix สำหรับ Ensemble
    plt.figure(figsize=(14, 12))
    cm = confusion_matrix(y_true, y_pred)
    ax = sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
    plt.title('Ensemble Model Confusion Matrix', fontsize=18, pad=20)
    plt.xlabel('Predicted Label', fontsize=14)
    plt.ylabel('True Label', fontsize=14)
    plt.xticks(rotation=45, ha='right', fontsize=12)
    plt.yticks(fontsize=12)
    plt.tight_layout()
    plt.savefig('ensemble_confusion_matrix.png', dpi=300)
    plt.show()
    
    # เปรียบเทียบความแม่นยำของโมเดลทั้งหมด
    plt.figure(figsize=(12, 8))

    # ทำนายด้วยแต่ละโมเดล
    cnn_rnn_preds = trained_cnn_rnn_model.predict(X_test)
    cnn_rnn_acc = accuracy_score(np.argmax(y_test, axis=1), np.argmax(cnn_rnn_preds, axis=1))

    resnet_preds = trained_resnet_model.predict(X_test)
    resnet_acc = accuracy_score(np.argmax(y_test, axis=1), np.argmax(resnet_preds, axis=1))

    # ข้อมูลสำหรับกราฟแท่ง
    models = ['CNN+RNN', 'ResNet', 'Ensemble']
    accuracies = [
        cnn_rnn_acc * 100, 
        resnet_acc * 100,
        accuracy * 100
    ]

    # เพิ่มค่าบนแท่ง
    for bar in bars:
        height = bar.get_height()
        plt.text(
            bar.get_x() + bar.get_width()/2.,
            height + 1,
            f'{height:.1f}%',
            ha='center', va='bottom', fontsize=12
        )
    
    plt.title('Model Comparison: Accuracy', fontsize=16)
    plt.xlabel('Model', fontsize=14)
    plt.ylabel('Accuracy (%)', fontsize=14)
    plt.ylim(0, 105)
    plt.grid(axis='y', linestyle='--', alpha=0.6)
    plt.savefig('model_comparison.png', dpi=300)
    plt.show()
    
    print("\nการฝึกสอนและทดสอบโมเดลเสร็จสิ้น")
    def preprocess_audio_file(audio_path, model_dir="Model"):
    try:
        # สกัดคุณลักษณะจากไฟล์เสียง
        features = extract_advanced_features(audio_path)
        if features is None:
            return None
        
        # แปลงเป็น numpy array และเพิ่มมิติแบตช์
        features = np.expand_dims(features, axis=0)
        
        return features
    except Exception as e:
        print(f"เกิดข้อผิดพลาดในการประมวลผลไฟล์เสียง: {str(e)}")
        return None

def predict_accent(audio_path, model_dir="Model"):
    """
    ทำนายสำเนียงจากไฟล์เสียง
    """
    try:
        # โหลด label encoder
        label_encoder = joblib.load(os.path.join(model_dir, 'label_encoder.pkl'))
        
        # โหลดโมเดล ensemble
        models = []
        for i in range(3):
            model_path = os.path.join(model_dir, f'accent_ensemble_model_{i+1}.h5')
            if os.path.exists(model_path):
                models.append(load_model(model_path))
            else:
                print(f"ไม่พบไฟล์โมเดล {model_path}")
        
        if len(models) == 0:
            # ถ้าไม่มีโมเดล ensemble ให้ใช้โมเดล CNN+RNN แทน
            model_path = os.path.join(model_dir, 'accent_cnn_rnn_model.h5')
            if os.path.exists(model_path):
                model = load_model(model_path)
                models = [model]
            else:
                raise FileNotFoundError("ไม่พบไฟล์โมเดล")
        
        # ประมวลผลไฟล์เสียง
        features = preprocess_audio_file(audio_path, model_dir)
        if features is None:
            return "ไม่สามารถประมวลผลไฟล์เสียงได้"
        
        # ทำนาย
        if len(models) > 1:
            # ใช้ ensemble
            predictions = []
            for model in models:
                pred = model.predict(features)
                predictions.append(pred)
            pred_probs = np.mean(predictions, axis=0)[0]
        else:
            # ใช้โมเดลเดียว
            pred_probs = models[0].predict(features)[0]
        
        # จัดอันดับผลการทำนาย
        top_indices = np.argsort(pred_probs)[::-1]
        
        # แสดงผลลัพธ์
        results = []
        for i, idx in enumerate(top_indices[:3]):  # แสดง 3 อันดับแรก
            accent = label_encoder.classes_[idx]
            confidence = pred_probs[idx] * 100
            results.append((accent, confidence))
        
        return results
    except Exception as e:
        print(f"เกิดข้อผิดพลาดในการทำนาย: {str(e)}")
        return "เกิดข้อผิดพลาดในการทำนาย"
if __name__ == "__main__":
    main()

IndentationError: expected an indented block after function definition on line 637 (3182720844.py, line 638)